# EQR Analysis

By 503 Team 4

## Background

A bank's equity capital ratio, similar to a bank's tier 1 capital ratio, is the amount of a bank's equity as a measure of its total capital. It is a measure of a bank's financial strength and ability to withstand economic downturns and absorb losses. The FDIC often requires a minimum 4% equity capital ratio for banks that it insures in order to allow the bank to become less reliant on debt and increase shareholder satisfaction. The equity capital ratio is an interesting figure to help project the stability of a bank. 

## Code

In [ ]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import altair as alt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

# alt.data_transformers.enable('json')
alt.data_transformers.enable('default',max_rows=None)

In [ ]:
#load data

csvs = []

file_path = '../data/quarterly_financials/'
for file in os.listdir(file_path):
    if file.endswith('.csv'):
        tmp_file = pd.read_csv(os.path.join(file_path, file))
        csvs.append(tmp_file)
df = pd.concat(csvs, ignore_index=True)

In [ ]:
#preprocessing
df['REPDTE'] = df['REPDTE'].astype(str)
#add year and quarter
df['YEAR'] = df['REPDTE'].str[:4]
df['QUARTER'] = ''

quarters_dict = {'0331': 'Q1', '0630': 'Q2', '0930': 'Q3', '1231': 'Q4'}
df['QUARTER'] = df['REPDTE'].str[-4:].map(quarters_dict).fillna('ERROR')

df['YEAR'] = df['YEAR'].astype(str)
df['QUARTER'] = df['QUARTER'].astype(str)
df['QUARTER_YEAR'] = df['YEAR'] + ' ' + df['QUARTER']

## Top and Bottom 20 Histogram

In [ ]:
df['COUNT'] = 1
df_eqr_overall = df.groupby('CERT').agg({'EQR' : 'mean', 'NAME': 'first', 'COUNT': 'sum', 'STNAME': 'first', 'CERT': 'first'})
df_eqr_overall = df_eqr_overall[df_eqr_overall['COUNT'] >= 100]

In [ ]:
mean_value = df_eqr_overall['EQR'].mean()
print(df_eqr_overall['EQR'].describe())

largest = df_eqr_overall.nlargest(20, 'EQR')
smallest = df_eqr_overall.nsmallest(20, 'EQR')
plot_eqr_df = pd.concat([largest, smallest], ignore_index=True)

mean_row = pd.DataFrame({'EQR': [mean_value], 'NAME' : ['MEAN']})
plot_eqr_df = pd.concat([plot_eqr_df, mean_row], ignore_index=True)
plot_eqr_df = plot_eqr_df.sort_values(by='EQR', ascending=True)

In [ ]:
selection = alt.selection_single(fields=['NAME'], name='Random')
color = alt.condition(selection,
                      alt.value('#118C4F'),
                      alt.value('#FFB90D'))

chart1 = alt.Chart(plot_eqr_df).mark_bar().encode(
    x=alt.X('NAME', sort=alt.EncodingSortField('EQR', order='ascending')),
    y='EQR',
    color=color,
    tooltip=['NAME', 'STNAME:N', 'EQR']
).properties(
    title='20 Largest and Smallest Banks by EQR in a Quarter, with Overall Average'
).add_selection(selection)
chart1.encoding.x.title = 'Bank Name (MEAN denotes overall average)'
chart1.encoding.y.title = 'Equity Capital Ratio (EQR)'

In [ ]:
plot_eqr_df_2 = df[df['CERT'].isin(plot_eqr_df['CERT'])]
plot_eqr_df_2 = plot_eqr_df_2.groupby(['CERT', 'QUARTER', 'YEAR']).agg({'QUARTER_YEAR': 'first', 'EQR': 'mean', 'NAME': 'first'}).reset_index()

line = (alt.Chart(plot_eqr_df_2)
         .mark_line(strokeWidth=5, color='#118C4F')
         .encode(x='QUARTER_YEAR:N',
                 y=alt.Y('mean(EQR):Q', scale=alt.Scale(domain=[0,100])),
                 color='NAME:N',
                 tooltip=['NAME', 'QUARTER_YEAR','EQR']
                 )
         ).transform_filter(selection)
line.title='Average EQR by Quarter over Past 30 Years'
line.encoding.x.title='Year and Quarter'
line.encoding.y.title='Equity Capital Ratio (EQR)'
chart1 & line


### Analysis

For this portion of the analysis, I chose to examine banks that represent the top and bottom 20 in average EQR by quarter. For this analysis, I filtered out any banks that do not appear in the data for more than 100 quarters (roughly 80% of the quarters). This is to eliminate any potential outliers in the data. The mean value does not all observations in the data; it only includes all banks with more than 100 quarters of data since Q1 1992. Looking at the histogram, we notice that the average EQR of 11.22 is far closer to the 20 banks that have the lowest equity capital ratio compared to the 20 banks that have the highest equity capital ratio. Looking at the 5 number summary, we see that the average (11.22) is lower than the median (10.27), indicating that there are a few banks with a far higher equity capital ratio than the majority of the rest of the banks in the FDIC database. When we more closely examine the banks that fall in the range of top or bottom 20, we see that there is no consistent state across these banks. It is noteworthy that Michigan has 2 of the 3 highest banks in terms of EQR. Another noteworthy observation is that banks in Ohio, South Dakota, and North Carolina only make up 3 of the top 20 banks in average equity capital ratio. Moreover, none of the largest banks in terms of total assets are present on this list. 

In [ ]:
top_banks = ["JPMORGAN CHASE BANK NA", "BANK OF AMERICA NA", "WELLS FARGO BANK NA", "CITIBANK NATIONAL ASSN"]
df_banks = df[df['NAME'].isin(top_banks)]

df_banks = df_banks.groupby(['CERT', 'QUARTER', 'YEAR']).agg({'QUARTER_YEAR': 'first', 'EQR': 'mean', 'NAME': 'first'}).reset_index()

line1 = (alt.Chart(df_banks)
         .mark_line(strokeWidth=5, color='#118C4F')
         .encode(x='QUARTER_YEAR:N',
                 y=alt.Y('mean(EQR):Q', scale=alt.Scale(domain=[6,35])),
                 color='NAME:N',
                 tooltip=['NAME', 'QUARTER_YEAR','EQR']
                 )
         )
line1.title='Average EQR by Quarter over Past 30 Years'
line1.encoding.x.title='Year and Quarter'
line1.encoding.y.title='Equity Capital Ratio (EQR)'

## Quarters Line Graph

In [ ]:
df_quarters = df.groupby(['QUARTER', 'YEAR']).agg({'QUARTER_YEAR': 'first', 'EQR': 'mean'}).reset_index()

line2 = (alt.Chart(df_quarters)
         .mark_line(strokeWidth=5, color='#118C4F')
         .encode(x='QUARTER_YEAR:N',
                 y=alt.Y('EQR:Q', scale=alt.Scale(domain=[6,15])),
                 tooltip=['QUARTER_YEAR','EQR'])
         )
line2.title='Average EQR by Quarter over Past 30 Years'
line2.encoding.x.title='Year and Quarter'
line2.encoding.y.title='Equity Capital Ratio (EQR)'
line1 & line2

We see based on the line graph above that, for the most part, the 4 major banks maintain a relatively consistent equity capital ratio. After 2004, which is the first quarter that JP Morgan Chase Bank NA is present in this data, we notice that none of the banks ever reach an equity capital ratio higher than 13, and for the most part, they have all reached a similar value between 8 and 10 since 2020. The only time any of these banks record an equity capital ratio that is well above the cumulative average is prior to the turn of the 20th century, when Bank of America NA records several quarters with an equity capital ratio over 20. Comparing the four largest banks to the overall average for all FDIC-insured banks present in the data, we see that, for the most part, the four largest banks are either identical to, or slightly below, the overall average equity capital ratio over the past 2 decades. This is due to the differing immediate needs between large national banks, such as Wells Fargo and Bank of America, and smaller, more localized banks. There are a few reasons for the diffence in focus. The first is that larger banks often have greater access to capital markets, and are thus less reliant on using their own capital to fund everyday business operations. Therefore, they do not necessarily need to maintain a high equity capital ratio. The next reason is because smaller banks often take a more conservative stance on risk management, and since equity capital ratio is a way to quantify a bank's risk management, it can in part explain why a smaller bank's equity capital ratio is higher, on average. Finally, the difference can be explained by differences in regulatory laws and the market/customer base that a bank is typically functioning in. Nevertheless, the FDIC does set regulations for a minimum equity capital ratio, so even the largest banks in the industry need to ensure that they have a sufficient equity capital ratio and are thus in a stronger financial position in the event of an economic downturn. While larger banks have more resources than smaller banks and thus have more methods of improving their financial position to respond to potential problems, they still must maintain sufficient equity.

## EQR and Assets: Small Counts

In [ ]:
df_eqr_year = df.groupby(['CERT', 'YEAR']).agg({'EQR': 'mean', 'NAME': 'first', 'CITY': 'first', 'STNAME': 'first', 'COUNT': 'sum', 'ASSET': 'mean', 'LIAB': 'mean', 'EQ': 'mean'})

In [ ]:
density_heatmap_df = df_eqr_year[df_eqr_year['ASSET'] <= 500000]

fig = px.density_heatmap(
    density_heatmap_df,
    x="ASSET",
    y="EQR",
    marginal_x="histogram",
    marginal_y="histogram",
    color_continuous_scale=px.colors.sequential.Viridis,
    nbinsx=50,
    nbinsy=1000,
    labels=dict(ASSET="Sum of Assets Owned", EQR="Equity Capital Ratio (EQR)"),
    title="Joint distribution of Average Assets Owned and Average EQR per Year",
    width=650,
    height=650,
    range_x = [1,500000],
    range_y = [0,15]
)

fig.show()

This plot attempts to look at smaller banks in terms of mean total assets owned over all quarters and forecast their average equity capital ratio. This heatmap contains over 200,000 unique banks, all with average total assets owned below 500,000. Based on the values of the heatmap, we see that the majority of these banks hold a small amount of assets, typically below 100,000. Nevertheless, most banks that make up this proportion maintain an equity capital ratio between 8 and 12, which is consistent relative to the average that we have previously examined. 

## Bubble Plot

In [ ]:
df_scatter = df.groupby(['CERT','YEAR'])
df_scatter = df.sort_values('NAME')
df_scatter = df_scatter[df_scatter['EQR']>=0]
df_scatter = df_scatter.sort_values('YEAR')

fig = px.scatter(
    df_scatter,
    x='EQ',
    y='EQR',
    animation_frame='YEAR',
    animation_group='NAME',
    size='EQR',
    color='BKCLASS',
    hover_name='NAME',
    template='plotly_white',
    range_color=[-100,100],
    size_max=45
)
fig.update_layout(
    title = 'Bubble Plot of Equity Capital and Equity Capital Ratio by Year',
    xaxis_title = 'Equity Capital (EQ)',
    yaxis_title = 'Equity Capital Ratio (EQR)',
    legend_title_text = 'Bank Class'
)
fig.show()